In [1]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [2]:
df = pd.read_csv("raw_ksi_data.csv")

In [3]:
df.shape

(18957, 54)

In [4]:
df.sample(2)

,OBJECTID,INDEX,ACCNUM,DATE,TIME,STREET1,STREET2,OFFSET,ROAD_CLASS,DISTRICT,...,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,x,y
1248,1249,3521578,942571.0,11/3/2006 10:00:00 AM,1154,4575 KENNEDY Rd,NaN,NaN,Major Arterial,Scarborough,...,NaN,NaN,NaN,116,Steeles,116,Steeles (116),D42,636181.4925,4.853523e+06
14139,14140,80788908,NaN,11/24/2017 10:00:00 AM,840,VEROBEACH BLVD,INDRIO RD,NaN,Local,Etobicoke York,...,NaN,NaN,NaN,22,Humbermede,22,Humbermede (22),D31,617398.8856,4.843781e+06


In [5]:
df.drop(['ACCNUM','OBJECTID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OFFSET', 'ROAD_CLASS', 'ACCLOC', 'INVAGE', 'INITDIR'], axis=1, inplace=True)

In [6]:
df.drop(['VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND', 'PEDTYPE', 'PEDACT', 'CYCLISTYPE', 'CYCACT', 'CYCCOND', 'PEDESTRIAN', "TRSN_CITY_VEH"], axis=1, inplace=True)

In [7]:
df.drop(['EMERG_VEH', 'PASSENGER', 'REDLIGHT', 'DISABILITY', 'DIVISION', 'SPEEDING', 'PEDCOND', 'AG_DRIV','x', 'y' ], axis=1, inplace=True)

In [8]:
df.drop(['CYCLIST', 'IMPACTYPE', 'INVTYPE', 'INDEX', 'TRAFFCTL', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'ALCOHOL'], axis=1, inplace=True)

In [9]:
df.shape

(18957, 15)

In [10]:
df.sample(2)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158
7170,7/19/2011 8:00:00 AM,1925,PARKSIDE DR,ALGONQUIN AVE,Toronto and East York,43.643145,-79.45549,Clear,Daylight,Dry,Non-Fatal Injury,Major,NaN,87,High Park-Swansea
5181,10/23/2009 8:00:00 AM,715,FINCH AVE E,BLACKBIRD GT,Scarborough,43.816945,-79.22429,Rain,Dark,Wet,Fatal,Minor,NaN,144,Morningside Heights


In [11]:
df['TIME'] = df['TIME'].astype(str).str.zfill(4)
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y %I:%M:%S %p')

# Extract DOW and HOUR and YEAR
df['DOW'] = df['DATE'].dt.day_name() 
df['HOUR'] = df['TIME'].str[:2].astype(int)
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

In [12]:
df.sample(3)

,DATE,TIME,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,RDSFCOND,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH
136,2006-02-16 10:00:00,2159,DOVERCOURT RD,NORTHUMBERLAND ST,Toronto and East York,43.662545,-79.429790,Rain,Dark,Wet,Non-Fatal Injury,NaN,NaN,172,Dovercourt Village,Thursday,21,2006,2
12933,2016-09-08 08:00:00,0037,MELFORD DR,TAPSCOTT RD,Scarborough,43.810794,-79.236697,Rain,"Dark, artificial",Wet,Non-Fatal Injury,NaN,NaN,145,Malvern West,Thursday,0,2016,9
3890,2008-09-27 08:00:00,2025,BIRCHMOUNT RD,LAWRENCE AVE E,Scarborough,43.747545,-79.284790,Rain,Dark,Wet,Non-Fatal Injury,Minimal,NaN,126,Dorset Park,Saturday,20,2008,9


In [13]:
df["INJURY"].unique()

array(['Major', 'Minor', nan, 'Fatal', 'Minimal'], dtype=object)

In [14]:
df["INJURY"].value_counts()

INJURY
Major      6445
Minor      1479
Minimal    1160
Fatal       976
Name: count, dtype: int64

In [15]:
df['INJURY'] = df['INJURY'].fillna('Minimal')

In [16]:
df["ACCLASS"].unique()

array(['Non-Fatal Injury', 'Fatal', 'Property Damage O', nan],
      dtype=object)

In [17]:
df["ACCLASS"].value_counts()

ACCLASS
Non-Fatal Injury     16268
Fatal                 2670
Property Damage O       18
Name: count, dtype: int64

In [18]:
df['ACCLASS'] = df['ACCLASS'].replace('Property Damage O', 'Non-Fatal Injury')
df['ACCLASS'] = df['ACCLASS'].fillna('Non-Fatal Injury')

In [19]:
df['SEVERE_COLLISION'] = (
    (df['INJURY'].isin(['Major', 'Fatal'])) | 
    (df['ACCLASS'] == 'Fatal')
).astype(int)

In [20]:
df['FATAL_NO'] = df['FATAL_NO'].fillna(0)

In [21]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear                     16373
Rain                       1976
Snow                        356
Other                        98
Fog, Mist, Smoke, Dust       52
Freezing Rain                47
Drifting Snow                21
Strong wind                  10
Name: count, dtype: int64

In [22]:
df['VISIBILITY'] = df['VISIBILITY'].replace({
    'Other': 'Clear',
    'Freezing Rain': 'Rain',
    'Strong wind': 'Rain',
    'Fog, Mist, Smoke, Dust': 'Rain',
    'Drifting Snow': 'Snow'
})

In [23]:
df["VISIBILITY"].value_counts()

VISIBILITY
Clear    16471
Rain      2085
Snow       377
Name: count, dtype: int64

In [24]:
df["LIGHT"].value_counts()

LIGHT
Daylight                10779
Dark                     3746
Dark, artificial         3552
Dusk                      253
Dusk, artificial          253
Daylight, artificial      151
Dawn                      112
Dawn, artificial          101
Other                       6
Name: count, dtype: int64

In [25]:
df['LIGHT'] = df['LIGHT'].replace({
    'Daylight, artificial': 'Daylight',
    'Dark, artificial': 'Dark',
    'Dusk, artificial': 'Daylight',
    'Dawn, artificial': 'Daylight',
    'Other': 'Daylight'
})

In [26]:
df["LIGHT"].value_counts()

LIGHT
Daylight    11290
Dark         7298
Dusk          253
Dawn          112
Name: count, dtype: int64

In [27]:
df.rename(columns={'RDSFCOND': 'ROAD_CONDITION'}, inplace=True)

In [28]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry                     15231
Wet                      3140
Loose Snow                174
Other                     147
Slush                     102
Ice                        78
Packed Snow                44
Loose Sand or Gravel       11
Spilled liquid              1
Name: count, dtype: int64

In [29]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace({
    'Spilled liquid': 'Wet',
    'Loose Snow': 'Snow_Ice',
    'Packed Snow': 'Snow_Ice',
    'Slush': 'Snow_Ice',
    'Ice': 'Snow_Ice',
    'Loose Sand or Gravel': 'Dry',
    'Other': 'Dry'
})

In [30]:
df["ROAD_CONDITION"].value_counts()

ROAD_CONDITION
Dry         15389
Wet          3141
Snow_Ice      398
Name: count, dtype: int64

In [31]:
df['DATE'] = pd.to_datetime(df['DATE']).dt.date

In [32]:
df.drop(['TIME'], axis=1, inplace=True)#since hour col, exact time not needed

In [33]:
df.sample(5)

,DATE,STREET1,STREET2,DISTRICT,LATITUDE,LONGITUDE,VISIBILITY,LIGHT,ROAD_CONDITION,ACCLASS,INJURY,FATAL_NO,HOOD_158,NEIGHBOURHOOD_158,DOW,HOUR,YEAR,MONTH,SEVERE_COLLISION
2503,2007-09-04,BAYVIEW AVE,SUNNYDENE CRES,North York,43.719745,-79.378690,Clear,Daylight,Dry,Non-Fatal Injury,Major,0.0,41,Bridle Path-Sunnybrook-York Mills,Tuesday,8,2007,9,1
7745,2011-12-10,SPADINA AVE,NASSAU ST,Toronto and East York,43.655845,-79.399190,Clear,Daylight,Dry,Non-Fatal Injury,Major,0.0,78,Kensington-Chinatown,Saturday,16,2011,12,1
10569,2014-03-29,455 MILNER AVE,NaN,Scarborough,43.791693,-79.228359,Clear,Dark,Dry,Non-Fatal Injury,Major,0.0,146,Malvern East,Saturday,2,2014,3,1
4859,2009-08-01,ISLINGTON AVE,THE WESTWAY,Etobicoke York,43.688645,-79.542890,Clear,Daylight,Dry,Non-Fatal Injury,Minor,0.0,6,Kingsview Village-The Westway,Saturday,6,2009,8,0
18502,2023-06-28,LAKE SHORE BV E,LOWER JARVIS ST,Toronto and East York,43.645548,-79.369792,Clear,Daylight,Dry,Non-Fatal Injury,Minimal,0.0,166,St Lawrence-East Bayfront-The Islands,Wednesday,12,2023,6,0


In [34]:
def categorize_hour(hour):
    if 0 <= hour < 8:
        return 'Late Night'
    elif 8 <= hour < 14:
        return 'Morning_Afternoon'
    elif 14 <= hour < 19:
        return 'Evening'
    else:
        return 'Night'

In [35]:
def categorize_season(month):
    if month in [12, 1, 2]:
        return 'Winter'
    elif month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    else:
        return 'Fall'

In [36]:
df['TIME_OF_DAY'] = df['HOUR'].apply(categorize_hour)
df['SEASON'] = df['MONTH'].apply(categorize_season)

In [37]:
df['ROAD_CONDITION'] = df['ROAD_CONDITION'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 1), 'ROAD_CONDITION'] = 'Wet'
df.loc[df['ROAD_CONDITION'].isna() & (df['SEVERE_COLLISION'] == 0), 'ROAD_CONDITION'] = 'Dry'

In [38]:
df['LIGHT'] = df['LIGHT'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 1), 'LIGHT'] = 'Dark'
df.loc[df['LIGHT'].isna() & (df['SEVERE_COLLISION'] == 0), 'LIGHT'] = 'Daylight'

In [39]:
df['VISIBILITY'] = df['VISIBILITY'].replace(['', ' ', 'nan', 'NaN'], np.nan)
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 1), 'VISIBILITY'] = 'Rain'
df.loc[df['VISIBILITY'].isna() & (df['SEVERE_COLLISION'] == 0), 'VISIBILITY'] = 'Clear'

In [40]:
df.shape

(18957, 21)

In [41]:
df.to_csv('ksi_collisions.csv', index=False)